# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [172]:
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common import NoSuchElementException
from datetime import datetime


In [173]:
navegador = webdriver.Chrome()



### Buscando a cotação do Dólar

In [174]:


navegador.get('https://www.google.com/')

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
try:
    cotacao_dolar = navegador.find_element(By.XPATH, '/html/body/div[7]/div/div[10]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')
except NoSuchElementException:
    cotacao_dolar = navegador.find_element(By.XPATH, '/html/body/div[7]/div/div[11]/div[1]/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')

cotacao_dolar = float(cotacao_dolar.replace(',', '.'))
cotacao_dolar


5.2231

### Buscando a cotação do Euro

In [175]:
navegador.get('https://www.google.com/')
navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')

navegador.find_element(By.XPATH,'/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

try:
    cotacao_euro = navegador.find_element(By.XPATH, '/html/body/div[7]/div/div[10]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')
except NoSuchElementException:
    cotacao_euro = navegador.find_element(By.XPATH, '/html/body/div[7]/div/div[11]/div[1]/div[2]/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[3]/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')

cotacao_euro = float(cotacao_euro.replace(',', '.'))

### Buscando a cotação do Ouro

In [176]:
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element(By.XPATH, '/html/body/div[5]/div[1]/div/div/input[2]').get_attribute('value')

cotacao_ouro = float(cotacao_ouro.replace(',', '.'))
cotacao_ouro

288.75

### Finalizando a busca e atualizando a base de dados

In [177]:
import pandas as pd
navegador.quit()


In [187]:
cotacoes = pd.DataFrame({
    'Moeda': ['Dólar', 'Euro', 'Ouro'],
    'Cotação': [cotacao_dolar, cotacao_euro, cotacao_ouro]
})
print("Cotacoes atualizadas em: ", datetime.now().strftime('%H:%M:%S'))
cotacoes

Cotacoes atualizadas em:  03:20:20


,Moeda,Cotação
0,Dólar,5.2231
1,Euro,5.1111
2,Ouro,288.7500


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [180]:
produtos = pd.read_excel('Produtos.xlsx')
produtos

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.222300,5222.247777,1.40,7311.146888
1,Carro Renault,4500.00,Euro,5.109263,22991.684922,2.00,45983.369844
2,Notebook Dell,899.99,Dólar,5.222300,4700.017777,1.70,7990.030221
3,IPhone,799.00,Dólar,5.222300,4172.617700,1.70,7093.450090
4,Carro Fiat,3000.00,Euro,5.109263,15327.789948,1.90,29122.800901
5,Celular Xiaomi,480.48,Dólar,5.222300,2509.210704,2.00,5018.421408
6,Joia 20g,20.00,Ouro,288.750000,5775.000000,1.15,6641.250000


In [181]:
produtos.loc[produtos['Moeda'] == 'Dólar','Cotação'] = cotacao_dolar
produtos.loc[produtos['Moeda'] == 'Euro','Cotação'] = cotacao_euro
produtos.loc[produtos['Moeda'] == 'Ouro','Cotação'] = cotacao_ouro
produtos

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.2231,5222.247777,1.40,7311.146888
1,Carro Renault,4500.00,Euro,5.1111,22991.684922,2.00,45983.369844
2,Notebook Dell,899.99,Dólar,5.2231,4700.017777,1.70,7990.030221
3,IPhone,799.00,Dólar,5.2231,4172.617700,1.70,7093.450090
4,Carro Fiat,3000.00,Euro,5.1111,15327.789948,1.90,29122.800901
5,Celular Xiaomi,480.48,Dólar,5.2231,2509.210704,2.00,5018.421408
6,Joia 20g,20.00,Ouro,288.7500,5775.000000,1.15,6641.250000


In [182]:
produtos["Preço de Compra"] = produtos["Preço Original"] * produtos["Cotação"]
produtos["Preço de Venda"] = produtos["Preço de Compra"] * produtos["Margem"]
display(produtos)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.2231,5223.047769,1.40,7312.266877
1,Carro Renault,4500.00,Euro,5.1111,22999.950000,2.00,45999.900000
2,Notebook Dell,899.99,Dólar,5.2231,4700.737769,1.70,7991.254207
3,IPhone,799.00,Dólar,5.2231,4173.256900,1.70,7094.536730
4,Carro Fiat,3000.00,Euro,5.1111,15333.300000,1.90,29133.270000
5,Celular Xiaomi,480.48,Dólar,5.2231,2509.595088,2.00,5019.190176
6,Joia 20g,20.00,Ouro,288.7500,5775.000000,1.15,6641.250000


In [183]:
produtos.to_excel('Produtos.xlsx', index=False)

print("Planilha atualizada às", datetime.now().strftime('%H:%M:%S'))

Planilha atualizada às 03:18:53


- Atualizando os preços e o cálculo do Preço Final

### Agora vamos exportar a nova base de preços atualizada